# Testing Qwen2.5-Coder 7B on Ollama with xpander.ai’s Function Calling

In [ ]:
pip install ollama 
pip install xpander-sdk

In [3]:
from xpander_sdk import XpanderClient
import ollama
## Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()
xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")
ollama_client = ollama.Client()
xpander_client = XpanderClient(api_key=xpanderAPIKey)
agent1 = xpander_client.agents.get(agent_id=xpanderAgentID)

In [ ]:
# Verify that you have ollama installed locally and can get responses before moving on
response = ollama_client.chat(model='qwen2.5-coder', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [4]:
print(agent1.get_tools())

[{'type': 'function', 'function': {'name': 'PgCrunchbaseLinkedInPerplexityDataEnrichment', 'description': "This operation enriches company data by integrating information from LinkedIn, Crunchbase, and Perplexity. Use it for comprehensive company analysis, retrieving employee details, or gathering the latest news and trends related to a specific company like Nvidia. It's particularly useful for market researchers, analysts, and business developers looking to enhance their understanding of a company's profile. IMPORTANT! Ensure to use body_params, query_params, path_params. These are crucial for correct function calling!", 'parameters': {'type': 'object', 'properties': {'queryParams': {'type': 'object', 'properties': {}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}, 'bodyParams': {'type': 'object', 'properties': {'inputTask': {'type': 'string', 'description': 'input user sub task based on the input query'}}, 'required': []}}, 'required': ['query_pa

In [5]:
memory = []
memory.append({"role": "system", "content": "You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer"})
print(memory[-1])
memory.append({"role": "user", "content": "Get news about qwen2.5-coder from HackerNews"})
print(memory[-1])
number_of_calls = 1
from xpander_sdk import LLMProvider

while True:
    llm_response = ollama_client.chat(
        model="qwen2.5-coder:14b",
        messages=memory,
        tools=agent1.get_tools()    
    )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    print(memory[-1])

    memory.append(llm_response['message'])
    print(memory[-1])
    if llm_response['message'].get('tool_calls'):
        print(llm_response['message'].get('tool_calls'))
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response,llm_provider=LLMProvider.OLLAMA)
        print(tools_to_run)
        tool_responses = agent1.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
            print(memory[-1])
    if (llm_response['message'].get('content')):
        if "##FINAL ANSWER##" in llm_response['message']['content']:
            break
    number_of_calls += 1
print(llm_response['message']['content'])

{'role': 'system', 'content': 'You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer'}
{'role': 'user', 'content': 'Get news about qwen2.5-coder from HackerNews'}
{'role': 'assistant', 'content': 'Step number: 1'}
{'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'PgCrunchbaseLinkedInPerplexityDataEnrichment', 'arguments': {'bodyParams': {}, 'pathParams': {}, 'queryParams': {'query': 'Qwen2.5-Coder', 'source': 'HackerNews'}}}}]}
[{'function': {'name': 'PgCrunchbaseLinkedInPerplexityDataEnrichment', 'arguments': {'bodyParams': {}, 'pathParams': {}, 'queryParams': {'query': 'Qwen2.5-Coder', 'source': 'HackerNews'}}}}]
{'role': 'tool', 'content': "system message: graph prompt group selected, ignore this and proceed with the user's request using new tools.", 'tool_call_id': ''}
{'role': 'assistant', 'content': 'Step number: 2'}
{'role': 'assistant', 'co

Known errors

While testing smaller LLMs for function calling, the following response was generated:

```json
{
  "model": "qwen2.5-coder",
  "created_at": "2024-11-15T22:15:25.093541Z",
  "message": {
    "role": "assistant",
    "content": "",
    "tool_calls": [
      {
        "function": {
          "name": "NotionContentSearchSearchPagesAndDatabases",
          "arguments": {
            "bodyParams": {},
            "pathParams": {},
            "queryParams": {"query": "Qwen-2.5-Coder"}
          }
        }
      }
    ]
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 4773907500,
  "load_duration": 29168833,
  "prompt_eval_count": 316,
  "prompt_eval_duration": 211000000,
  "eval_count": 123,
  "eval_duration": 4525000000
}
```

However, this output violates the defined function calling schema, as shown below:

```json
[
  {
    "type": "function",
    "function": {
      "name": "NotionContentSearchSearchPagesAndDatabases",
      "description": "Searches pages and databases across the Notion workspace...",
      "parameters": {
        "type": "object",
        "properties": {
          "bodyParams": { "type": "object", "properties": { ... } },
          "queryParams": { "type": "object", "properties": {}, "required": [] },
          "pathParams": { "type": "object", "properties": {}, "required": [] }
        },
        "required": ["bodyParams", "queryParams", "pathParams"]
      }
    }
  }
]
```

Key Issues Observed

1.	Incomplete Required Parameters:
	•	The schema mandates that bodyParams, queryParams, and pathParams must be populated and valid.
	•	In the LLM response:
	•	bodyParams is an empty object ({}) with no filters, sorting, or pagination fields.
	•	pathParams is also empty, violating the schema requirement.
	•	Only queryParams contains a single field (query), which is insufficient for the intended functionality.
2.	Schema Validation Failure:
	•	The response does not conform to the schema’s required directive, which specifies: `"required": ["bodyParams", "queryParams", "pathParams"]` While the fields exist, they are effectively invalid because they lack necessary data.


# Summary

Implications for Testing Smaller Models

When evaluating smaller LLMs for function calling, this bug highlights several challenges:
1. Lack of Contextual Understanding:
   - Smaller models may struggle to fully comprehend and populate complex schemas, especially when required fields have nested structures or multiple interdependent parameters.
2.	Limited Parameter Generation Capability:
	•	These models may fail to infer the importance of fields like filter or sort, defaulting to minimalistic responses (e.g., only populating queryParams).